In [1]:
from xcube.core.store import new_data_store
import os
import dask
from dask.distributed import Client, LocalCluster

In [ ]:
cluster = LocalCluster(
    n_workers=4,
    threads_per_worker=2,
    memory_limit="7GB",   # per worker
)
client = Client(cluster)

In [4]:
store = new_data_store("stac-pc-ardc")

In [5]:
from xcube_stac.utils import reproject_bbox

#bbox=[-6.3, 53.2, -3.2, 55.3]
#bbox=[-3.3,55.9,-2.3, 56.2] # Edinburgh
#bbox=[4.7,58.6,6.7, 60.6] # Stavanger
#bbox=[4.7,58.6,6.1, 59.6] # Stavanger
#bbox_wgs84 = [7.7, 55.5, 8.4, 56.1] #Esbjerg
#bbox_wgs84 = [4.3, 59.0, 5.7, 59.6] #north of Stavanger
bbox_wgs84 = [4.3, 58.0, 6.5, 59.6] #large Stavanger


crs_utm = "EPSG:32632"
bbox_utm = reproject_bbox(bbox_wgs84, "EPSG:4326", crs_utm)
bbox_utm

(222323.42250606988, 6431444.486300742, 358897.7685275072, 6616254.075772593)

In [6]:
store.list_data_ids()

['sentinel-2-l2a']

In [8]:
%%time
ds = store.open_data(
    "sentinel-2-l2a",
    #time_range=("2022-08-07", "2022-08-16"),
    time_range=("2023-06-07", "2023-06-16"),
    bbox=bbox_wgs84,
    spatial_res=10 / 111320,
    crs="EPSG:4326",
    asset_names=["B02", "B03", "B04", "B08", "SCL"],
    tile_size=(2048, 2048),
)
ds

CPU times: user 8min 42s, sys: 8.78 s, total: 8min 51s
Wall time: 9min 7s


<xarray.Dataset> Size: 45GB
Dimensions:      (time: 6, lon: 24492, lat: 17813)
Coordinates:
  * time         (time) datetime64[ns] 48B 2023-06-07T10:56:21.024000 ... 202...
  * lon          (lon) float64 196kB 4.3 4.3 4.3 4.3 4.3 ... 6.5 6.5 6.5 6.5 6.5
  * lat          (lat) float64 143kB 59.6 59.6 59.6 59.6 ... 58.0 58.0 58.0 58.0
    spatial_ref  int64 8B 0
Data variables:
    B02          (time, lat, lon) float32 10GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B03          (time, lat, lon) float32 10GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B04          (time, lat, lon) float32 10GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B08          (time, lat, lon) float32 10GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    SCL          (time, lat, lon) uint8 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    stac_item_ids:       {'2023-06-07T10:56:21.024000': ['S2A_MSIL2A_20230607...
    stac_catalog_url:    https://planetarycomputer.microsoft.com/api/stac/v1
    xcube_stac_version:  1.1.1

In [9]:
S3_USER_STORAGE_KEY = os.environ["S3_USER_STORAGE_KEY"]
S3_USER_STORAGE_SECRET = os.environ["S3_USER_STORAGE_SECRET"]
S3_USER_STORAGE_BUCKET = os.environ["S3_USER_STORAGE_BUCKET"]

In [10]:
from xcube.core.store import new_data_store
team_store = new_data_store("s3", 
                       root=S3_USER_STORAGE_BUCKET, 
                       storage_options=dict(anon=False, 
                                            key=S3_USER_STORAGE_KEY, 
                                            secret=S3_USER_STORAGE_SECRET))

list(team_store.get_data_ids())

['cmems_4km_2022080317.zarr',
 'cmems_4km_2023060717.zarr',
 'cmems_7km_2022080317.zarr',
 'cmems_7km_2023060717.zarr',
 'sen2l2a_edinburg_2022080712.zarr',
 'sen2l2a_esbjerg_2022080716.zarr',
 'sen2l2a_stavanger_2022080712_2.zarr',
 'sen2l2a_stavanger_2023060716.zarr',
 'sen2l2a_stavanger_2023060716_2.zarr']

In [11]:
%%time
team_store.write_data(ds,'sen2l2a_stavanger_large.zarr', replace=True)

/home/conda/users/30f6ffd2-1762860680-159-deepesdl-xcube-1.12.0/lib/python3.13/site-packages/xcube/core/store/fs/impl/dataset.py:200: UserWarning: Times can't be serialized faithfully to int64 with requested units 'seconds since 1970-01-01'. Resolution of 'milliseconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  data.to_zarr(
/home/conda/users/30f6ffd2-1762860680-159-deepesdl-xcube-1.12.0/lib/python3.13/site-packages/distributed/client.py:3374: UserWarning: Sending large graph of size 275.76 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/home/conda/users/30f6ffd2-1762860680-159-deepesdl-xcube-1.12.0/lib/

CPU times: user 7min 53s, sys: 40.9 s, total: 8min 34s
Wall time: 35min 17s


'sen2l2a_stavanger_large.zarr'

In [12]:
%%time
ds = team_store.open_data('sen2l2a_stavanger_large.zarr')
ds

CPU times: user 2.32 s, sys: 68.1 ms, total: 2.39 s
Wall time: 2.55 s


<xarray.Dataset> Size: 45GB
Dimensions:      (time: 6, lat: 17813, lon: 24492)
Coordinates:
  * time         (time) datetime64[ns] 48B 2023-06-07T10:56:21.024000 ... 202...
  * lat          (lat) float64 143kB 59.6 59.6 59.6 59.6 ... 58.0 58.0 58.0 58.0
  * lon          (lon) float64 196kB 4.3 4.3 4.3 4.3 4.3 ... 6.5 6.5 6.5 6.5 6.5
    spatial_ref  int64 8B ...
Data variables:
    B02          (time, lat, lon) float32 10GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B03          (time, lat, lon) float32 10GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B04          (time, lat, lon) float32 10GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B08          (time, lat, lon) float32 10GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    SCL          (time, lat, lon) uint8 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    stac_catalog_url:    https://planetarycomputer.microsoft.com/api/stac/v1
    stac_item_ids:       {'2023-06-07T10:56:21.024000': ['S2A_MSIL2A_20230607...
    xcube_stac_version:  1.1.1

In [13]:
%%time
team_store.write_data(ds,"sen2l2a.levels", num_levels=5)

CPU times: user 2min 2s, sys: 16.3 s, total: 2min 19s
Wall time: 8min 44s


'sen2l2a.levels'